In [ ]:
%pip install nltk
%pip install tweepy
%pip install pandas
%pip install wordcloud
%pip install textblob
%pip install seaborn
%pip install poyraz
%pip install spacy
%pip install sklearn
%pip install zemberek-python
%pip install trnltk
%pip install git+https://github.com/emres/turkish-deasciifier.git
%pip install transformers datasets
%pip install torch
%pip install tensorflow

In [326]:
#!/usr/bin/env python
#-*-coding:utf-8-*-
from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)
import logging
import tweepy
import json
import csv
import sys
import time
import re
import warnings
import string
import pandas as pd
import ast
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
from trnlp import TrnlpWord
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, TextClassificationPipeline
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cemke\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [473]:
class getTweets:
    def __init__(self,ckey,csecret,atoken,asecret):
        self.api=None
        self.consumer_key = ckey
        self.consumer_secret = csecret
        self.access_token = atoken
        self.access_token_secret = asecret
        
    def setAuth(self):
        try:
            self.auth = tweepy.OAuthHandler(self.consumer_key, self.consumer_secret)
            self.auth.set_access_token(self.access_token, self.access_token_secret)
            self.api = tweepy.API(self.auth)            
            print ("Bağlantı Yapıldı!")                           
        except tweepy.TweepError as err:
            print('Error: {}'.format(err))   
    
    def toDataFrame(tweets):
        try:
            print ("Veriler Kayıt Ediliyor...")
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', None)
            pd.set_option('display.max_colwidth', None)
            DataSet = pd.DataFrame()

            DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]
            DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]   
            #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
            DataSet['Tweet'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Degistirilmis'] = [tweet.text for tweet in tweets]
            DataSet['Tweet_Ceviri']=[tweet.text for tweet in tweets]            
            #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
                
            #DataSet['Dil'] = [tweet.lang for tweet in tweets]
            print ("Veriler Kayıt Edildi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def searchTweet(self,aranan):
        self.setAuth()
        try:                      
            print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
            cursor = tweepy.Cursor(self.api.search_tweets, q=aranan, result_type="new", lang='tr')
            results=[]
            for item in cursor.items(10):
                results.append(item)
            print ("Kelimeye Uygun Tweetler Sorgulandı!")
            DataSet = getTweets.toDataFrame(results)  
            DataSet.to_csv('tw.csv',index=False)
        except Exception as err:
            print('Error: {}'.format(err))
        finally:
            Dataset=pd.read_csv('tw.csv')                   
            getTweets.TweetClear(DataSet)
            getTweets.SentenceNormalization(DataSet)
            getTweets.TweetTranslate(DataSet)
            getTweets.KelimeAyristirma(DataSet) 
            getTweets.ToxicSentiment(DataSet)
            getTweets.TextblobSentiment(DataSet)

    def TextblobSentiment(DataSet):
        try:
            print ("Duygu Analizi Yapılıyor...")
            Veri=DataSet
            Veri["Tür"] = Veri["Tweet_Ayrı"].apply(getTweets.getPolarity)
            Veri["Yoğunluk"] = Veri["Tweet_Ayrı"].apply(getTweets.getSubjectivity)
            Veri.to_csv('textblob_eng_duygu.csv',index=False)
            print("Duygu Analizi Yapıldı!")                        
        except Exception as err:
            print('Error: {}'.format(err)) 
            
    """def TrSentiment(DataSet):
        try:
            print("Türkçe Duygu Analizi Yapılıyor...")
            for example,index in DataSet["Tweet_Degistirilmis"]:
                DataSet["TR_A"][index] = simple_token(example, sw=stopwords)
            print("Türkçe Duygu Analizi Yapıldı!")
        except Exception as err:
            print('Error: {}'.format(err)) """
                     
    def TweetTranslate(DataSet):
        try:
            data=DataSet
            print ("Tweetler Çeviriliyor...")
            for index in range(len(data["Tweet_Degistirilmis"])):
                Ceviri_Blob=TextBlob(data["Tweet_Degistirilmis"][index])
                Eng_Blob=Ceviri_Blob.translate(from_lang='tr', to='en')  
                data["Tweet_Ceviri"][index]=Eng_Blob.lower()
                
            print ("Tweetler Çevirildi!")                      
            return DataSet        
        except Exception as err:
            print('Error: {}'.format(err))
            
    def TweetClear(DataSet):
        try:
            print ("Tweetler Temizleniyor...")
            pattern = re.compile('http[s]?://(?:[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F]))+|#[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|$[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ]+|@[a-zA-Z0-9ğüşöçıİĞÜŞÖÇ_]+|[,.^_$*%-;鶯!?’"“”:=|…#]+|\n+|RT[\s]+|amp[\s]+|(^gt)')
            for index in range(len(DataSet["Tweet_Degistirilmis"])):        
                DataSet["Tweet_Degistirilmis"][index]=pattern.sub(' ', DataSet["Tweet_Degistirilmis"][index])    
                DataSet["Tweet_Degistirilmis"][index]=DataSet["Tweet_Degistirilmis"][index].lower()              
            DataSet["Tweet_Degistirilmis"]=DataSet["Tweet_Degistirilmis"].str.replace("[^\w\s]","")
            print ("Tweetler Temizlendi!")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def SentenceNormalization(DataSet):
        try:
            morphology = TurkishMorphology.create_with_defaults()
            normalizer = TurkishSentenceNormalizer(morphology)
            for data,index in DataSet["Tweet_Degistirilmis"].items():
                 DataSet["Tweet_Degistirilmis"][index]= normalizer.normalize(data)
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
            
    def KelimeAyristirma(DataSet):
        try:
            stop = stopwords.words('english')
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ceviri"].str.split()
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].apply(lambda x: [item for item in x if item not in stop])
            DataSet["Tweet_Ayrı"] = DataSet["Tweet_Ayrı"].str.join(" ")
            return DataSet
        except Exception as err:
            print('Error: {}'.format(err))
    
    def ToxicSentiment(DataSet):
        try:
            data=DataSet
            print("Toxic Cümle Analizi Yapılıyor...")
            model_path ="martin-ha/toxic-comment-model"
            tokenizer=AutoTokenizer.from_pretrained(model_path)
            model=AutoModelForSequenceClassification.from_pretrained(model_path)
            pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
            for examples, index in data["Tweet_Ceviri"].items():
                sonuc=pipeline(str(examples))
                if sonuc[0]['label']=="toxic":
                    data["Tweet_Toxic"][index]="Toxic"
                else:
                    data["Tweet_Toxic"][index]="Toxic değil"    
            print("Toxic Cümle Analizi Yapıldı!")
            return DataSet  
        except Exception as err:
            print('Error: {}'.format(err))
        
    def getSubjectivity(DataSet):
        try: 
            sub=TextBlob(DataSet).sentiment.subjectivity               
            if sub >=0.75:
                return "Kesinlikle"
            elif sub >=0.5 and sub <0.75:
                return "Yüksek İhtimal"
            elif sub >=0.25 and sub <0.50:
                return "Düşük İhtimal"
            else:
                return "Kesin Değil"
        except Exception as err:
            print('Error: {}'.format(err))
            
    def getPolarity(DataSet):
        try:
            pol=TextBlob(DataSet).sentiment.polarity
            if pol>0.1:
                return "Pozitif"
            elif pol<-0.1:
                return "Negatif"
            else:
                return "Nötr"
        except Exception as err:
            print('Error: {}'.format(err))

In [474]:
tw = getTweets( 
ckey = 'o7qidRODRWm5u07PtAWrCpSqf',
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV',
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc',
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'
)
aranan=input("Aranacak Kelimeyi Girin: ")
tw.searchTweet(aranan)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!
2023-01-12 03:31:25,805 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 9.799805402755737

Error: 'int' object has no attribute 'translate'
Tweetler Çeviriliyor...
Tweetler Çevirildi!
Toxic Cümle Analizi Yapılıyor...
Error: 'Tweet_Toxic'
Duygu Analizi Yapılıyor...
Duygu Analizi Yapıldı!


In [475]:
Veri=pd.read_csv('textblob_eng_duygu.csv')
Veri.head()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
0,♡🍋🍊,2023-01-11 23:54:02+00:00,RT @simaldiyebiri: Kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken…,kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken,burning a bulb in his head after the enlightenment of the rice shop to offer the offer lsşskdkskcodk this child aybiken,burning bulb head enlightenment rice shop offer offer lsşskdkskcodk child aybiken,Nötr,Kesin Değil
1,alfa,2023-01-11 23:31:22+00:00,@TamerAkyuz5 @ankara_kusu Yok edisonun bulduğu ampul allahin bahsettigi din var ve ikisini iyi pqzarliyorlar ed jina venim,yok edisonun bulduğu ampul allahin bahsettigi din var ve ikisini iyi pqzarliyorlar ed jina venim,the bulb found by the destruction of the destruction is the religion that god speaks and they are well pqzarlaşlar ed jina veinim,bulb found destruction destruction religion god speaks well pqzarlaşlar ed jina veinim,Nötr,Kesin Değil
2,ＥＬＡ,2023-01-11 23:20:59+00:00,RT @simaldiyebiri: Kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken…,kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken,burning a bulb in his head after the enlightenment of the rice shop to offer the offer lsşskdkskcodk this child aybiken,burning bulb head enlightenment rice shop offer offer lsşskdkskcodk child aybiken,Nötr,Kesin Değil


In [476]:
Veri.tail()

,Kullanıcı_Adı,Tweet_Tarih,Tweet,Tweet_Degistirilmis,Tweet_Ceviri,Tweet_Ayrı,Tür,Yoğunluk
0,♡🍋🍊,2023-01-11 23:54:02+00:00,RT @simaldiyebiri: Kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken…,kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken,burning a bulb in his head after the enlightenment of the rice shop to offer the offer lsşskdkskcodk this child aybiken,burning bulb head enlightenment rice shop offer offer lsşskdkskcodk child aybiken,Nötr,Kesin Değil
1,alfa,2023-01-11 23:31:22+00:00,@TamerAkyuz5 @ankara_kusu Yok edisonun bulduğu ampul allahin bahsettigi din var ve ikisini iyi pqzarliyorlar ed jina venim,yok edisonun bulduğu ampul allahin bahsettigi din var ve ikisini iyi pqzarliyorlar ed jina venim,the bulb found by the destruction of the destruction is the religion that god speaks and they are well pqzarlaşlar ed jina veinim,bulb found destruction destruction religion god speaks well pqzarlaşlar ed jina veinim,Nötr,Kesin Değil
2,ＥＬＡ,2023-01-11 23:20:59+00:00,RT @simaldiyebiri: Kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken…,kafasinda ampul yanmali bi aydinlanma yasadiktan sonra pilav dukkani acmayi teklif etmesi lsşskdkskcodk bu cocuk aybiken,burning a bulb in his head after the enlightenment of the rice shop to offer the offer lsşskdkskcodk this child aybiken,burning bulb head enlightenment rice shop offer offer lsşskdkskcodk child aybiken,Nötr,Kesin Değil


In [477]:
"""#Argo cümle
model_path ="martin-ha/toxic-comment-model"
tokenizer=AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path)
pipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)
text="bu cümle toksik bir cümle değil amınakoyayım"
b_text=TextBlob(text)
e_text=b_text.translate(from_lang='tr', to='en')  
sonuc=pipeline(str(e_text))
if sonuc[0]['label']=="toxic":
    print ("Toxic")
else:
    print ("Toxic değil")"""

'#Argo cümle\nmodel_path ="martin-ha/toxic-comment-model"\ntokenizer=AutoTokenizer.from_pretrained(model_path)\nmodel=AutoModelForSequenceClassification.from_pretrained(model_path)\npipeline=TextClassificationPipeline(model=model, tokenizer=tokenizer)\ntext="bu cümle toksik bir cümle değil amınakoyayım"\nb_text=TextBlob(text)\ne_text=b_text.translate(from_lang=\'tr\', to=\'en\')  \nsonuc=pipeline(str(e_text))\nif sonuc[0][\'label\']=="toxic":\n    print ("Toxic")\nelse:\n    print ("Toxic değil")'

In [478]:
#Eksik Cümle
unmasker=pipeline('fill-mask', model='xlm-roberta-base')
